### [백준 2004번 조합 0의 개수](https://www.acmicpc.net/problem/2004)

##### 시간제한: 2초, 메모리제한 128MB

### 문제

_nCm_의 끝자리 0의 개수를 출력하는 프로그램을 작성하시오.

### 입력

첫째 줄에 정수 n, m (0 <= m <= n <= 2,000,000,000, n != 0)이 들어온다

### 출력

첫째 줄에 _nCm_의 끝자리 0의 개수를 출력한다.

### 접근

어제 풀었던 팩토리얼 0의 개수를 사용할 수 있을 듯 하다.

_nCm_의 정의를 생각해보면, _nCm_ = _n!/m!(n-m)!_ 이므로

n!의 0의 개수 - m!의 0의 개수 - (n-m)!의 0의 개수를 출력하면 된다.

n!의 0의 개수를 먼저 구하는도중에 그보다 작은 수들에 대한것들도 한꺼번에 계산하면 m!과 n-m!은 이미 계산했던 내용을 가져다 사용하면 될 것이다.

In [2]:
inputLine = input().split(' ')
n = int(inputLine[0])
m = int(inputLine[1])

zero_fac = [-1] * (n+1)
zero_fac[0] = 0

two = 0
five = 0
for i in range(1, n+1):
    tmp = i
    while True:
        if tmp % 2 == 0:
            two += 1
            tmp /= 2
        if tmp % 5 == 0:
            five += 1
            tmp /= 5
        if tmp % 2 != 0 and tmp % 5 != 0:
            break
    if two >= 1 and five >= 1:
        zero_fac[i] = zero_fac[i-1] + min(two, five)
        two -= 1
        five -= 1
    else:
        zero_fac[i] = zero_fac[i-1]

print(zero_fac[n] - zero_fac[m] - zero_fac[n-m])

25 12
2


결과는 메모리초과.

아무래도 N의 범위가 2백만이다보니까 zero_fac list의 메모리 차지가 너무 심했던 것 같다.

해결방법은 리스트를 아예 만들지 않거나, 리스트 크기를 줄이는 방법.

n!의 0의 개수를 구할 때 나머지 것들도 구해지는 점과 zero_fac 에는 한 항의 차이를 두고 관계가 이루어지니 리스트를 사용하지 않는 방향을 택하자.

In [3]:
inputLine = input().split(' ')
n = int(inputLine[0])
m = int(inputLine[1])
zero_n = 0
zero_m = 0
zero_sub = 0

two = 0
five = 0
before = 0
for i in range(1, n+1):
    tmp = i
    while True:
        if tmp % 2 == 0:
            two += 1
            tmp /= 2
        if tmp % 5 == 0:
            five += 1
            tmp /= 5
        if tmp % 2 != 0 and tmp % 5 != 0:
            break
    if two >= 1 and five >= 1:
        before += min(two, five)
        two -= 1
        five -= 1

    if n == i:
        zero_n = before
    if m == i:
        zero_m = before
    if n-m == i:
        zero_sub = before

print(zero_n-zero_m-zero_sub)

25 12
2


결과는 시간초과.

현재 방식에서 바꿀 방법을 잘 모르겠으니 접근 방식 자체를 달리 해보자.

n!의 0의 개수를 구한다고 해서 이전 수들의 0의 개수를 다 구하지 말고 각각을 따로 구하자.

결국 소인수분해를 2와 5에 대해서만 하자.

In [1]:
def get_zero(N):
    two = 0
    five = 0
    for i in range(1, N+1):
        two_tmp = 1
        five_tmp = 1

        if i % 2 == 0:
            while i % (2**two_tmp) == 0:
                two_tmp += 1
            two += two_tmp -1
        if i % 5 == 0:
            while i % (5**five_tmp) == 0:
                five_tmp += 1
            five += five_tmp -1
    return min(two, five)

inputLine = input().split(' ')
n = int(inputLine[0])
m = int(inputLine[1])
print(get_zero(n) - get_zero(m) - get_zero(n-m))

25 12
2


결과는 다시 시간초과.

아무리 해도 생각이 안나서 솜장님의 블로그를 보았다.

내가 쓴 코드랑 접근 방식 자체는 같은데, 이를 구현하는 방법에 약간 차이가 있었다.


In [4]:
def countNum(N, num): 
    count = 0 
    divNum = num 
    while( N >= divNum): 
        count = count + (N // divNum) 
        divNum = divNum * num 
    return count

N!이 가지고 있는 num의 개수를 반환하는 함수이다.

확실히 내 코드와 비교해보면 솜장님의 코드는 와일문 한번, 제곱수가 N 보다 작아야하므로 상당히 적은 수로 루프가 돌지만

내 코드의 경우 1부터 N까지 돌아가며 인수분해를 일일히 다 해주다보니 O(n^2)으로 시간이 걸리는 듯 하다.

그럼 이제 내 코드에 위 알고리즘을 대입해보자.

In [6]:
def get_zero(N):
    two = 0
    five = 0
    
    N_ori = N
    divNum = 2
    while N >= divNum:
        two += N // divNum
        divNum *= 2
        
    N = N_ori
    divNum = 5
    while N >= divNum:
        five += N // divNum
        divNum *= 5
    return min(two, five)

inputLine = input().split(' ')
n = int(inputLine[0])
m = int(inputLine[1])
print(get_zero(n) - get_zero(m) - get_zero(n-m))

25 12
2


결과는 실패?!

좀 생각해보니 맨 처음에 생각했던 n!의 0의 개수 - m!의 0의 개수 - (n-m)!의 0의 개수

가 틀렸던 것이다.

간단하게는 m!(n-m)! 을 하는 과정에서, 즉 곱해지는 과정에서 0이 더 추가될 수도 있다는 것이다.

결국 다시 공식을 만들어보면,

min( n!의 2의 개수 - m!의 2의 개수 - (n-m)!의 2의 개수 , n!의 5의 개수 - m!의 5의 개수 - (n-m)!의 5의 개수 ) 가 되어야 한다.

결론적으로는 솜장님의 코드를 그대로 가져오는 것이 좋다.

In [7]:
def countNum(N, num): 
    count = 0 
    divNum = num 
    while N >= divNum:
        count += N // divNum
        divNum *= num 
    return count

inputLine = input().split(' ')
n = int(inputLine[0])
m = int(inputLine[1])
print(min((countNum(n, 2) - countNum(m, 2) - countNum(n-m, 2)), (countNum(n, 5) - countNum(m, 5) - countNum(n-m, 5))))

25 12
2


결과는 성공.

하지만 역시 솜장님의 코드를 가져온 것이 아쉽다.